In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

In [5]:
X, y = load_breast_cancer(return_X_y= True)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [33]:
n, d = X_train.shape

phi = np.mean(y_train)

mask1 = (y_train == 1)
mask0 = ~mask1
n1 = mask1.sum()
n0 = mask0.sum()

mu1 = X_train[mask1].sum(axis=0)/ n1
mu0 = X_train[mask0].sum(axis=0)/ n0

X_centered = np.empty_like(X_train)
X_centered[mask1] = X_train[mask1] - mu1
X_centered[mask0] = X_train[mask0] - mu0

Sigma = (X_centered.T @ X_centered)/n

eps = 1e-8
Sigma += eps* np.eye(Sigma.shape[0])



In [34]:
def h0(X, phi, mu1, mu0, Sigma):
    inv_Sigma = np.linalg.inv(Sigma)
    det_Sigma = np.linalg.det(Sigma)

    norm_const = (2*np.pi)**(X.shape[1]/2) * np.sqrt(det_Sigma)

    def gaussian_pdf(x, mu):
        diff = x - mu
        return np.exp(-0.5 * diff.T @ inv_Sigma @ diff)/ norm_const

    probs = []
    for x in X:
        p1 = phi*gaussian_pdf(x, mu1)
        p0 = (1-phi)*gaussian_pdf(x, mu0)
        probs.append(p1/(p1+p0))
    
    return np.array(probs)

In [37]:
probs = h0(X_test, phi, mu1, mu0, Sigma)
y_preds = (probs>=0.5).astype(int)

accuracy = np.mean(y_test==y_preds)
print(accuracy)

0.956140350877193
